# Querying the Database Collection

In [1]:
#Loading environment variables

import os

WEAVIATE_CLUSTER_URL = os.getenv("WEAVIATE_CLUSTER_URL")
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

## Connect to Weaviate Instance

In [18]:
import weaviate

#Connecting to WCS instance

client = weaviate.connect_to_wcs(
    cluster_url=WEAVIATE_CLUSTER_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_API_KEY),
    headers={
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
        "X-Cohere-Api-Key": COHERE_API_KEY
    }
)

client.is_ready()

True

## Fetch Collection

In [19]:
questions = client.collections.get("Questions")

## Vector Search

In [23]:
response = questions.query.near_text(
    query="wild animals",
    limit=5
)

for item in response.objects:
    print(item.properties["answer"])
    print(item.properties["question"])

Bull moose
When he began his Progressive Party, Teddy Roosevelt said he felt as fit as one of these animals
ivory
This gemlike carving material was obtained from walruses as well as elephants
a Yahoo
In "Gulliver's Travels", Swift described this type of creature as "the most unteachable of all brutes"
hyena
It's the amused African mammal heard <a href="http://www.j-archive.com/media/2006-12-14_J_10.mp3">here</a>
Mongoose
The Indian species of this cobra killer can be tamed when young


## Search with Filters

In [25]:
import weaviate.classes.query as Query

response = questions.query.near_text(
    query="wild animals",
    limit=5,
    filters=Query.Filter.by_property("points").greater_than(500)
)

for item in response.objects:
    print(item.properties["answer"])
    print(item.properties["question"])
    print(item.properties["points"])

a Yahoo
In "Gulliver's Travels", Swift described this type of creature as "the most unteachable of all brutes"
2000
hyena
It's the amused African mammal heard <a href="http://www.j-archive.com/media/2006-12-14_J_10.mp3">here</a>
1000
Amphibious
An aircraft that can take off & touch down on water or land is called this, like a class of animals
600
wildebeest
What else is gnu, but that the gnu is also called this, from Dutch words
1600
the Ice Age
The biggest bear ever, an 11-foot 1,800-pounder, became extinct around the time this age ended 12,000 years ago
800


## Hybrid Search

In [26]:
response = questions.query.hybrid(
    query="wild african animals",
    alpha=0.5, # %of vector search
    auto_limit=1,
)

for item in response.objects:
    print(item.properties["answer"])
    print(item.properties["question"])
    print(item.properties["points"])

hyena
It's the amused African mammal heard <a href="http://www.j-archive.com/media/2006-12-14_J_10.mp3">here</a>
1000
gorilla
Even in the wild, this largest of apes takes a midday nap
100
Rhinoceros
The "black" species of this large horned mammal can grasp twigs with its upper lip
400
Mongoose
The Indian species of this cobra killer can be tamed when young
300


## RAG

In [28]:
# client.generate
response = questions.generate.near_text(
    query="wild animals",
    limit=3,
    single_prompt="Please give me a good question to answer this {answer}"
)

for item in response.objects:
    print(item.properties["question"])
    print(item.properties["answer"])
    print(item.generated, "\n")

When he began his Progressive Party, Teddy Roosevelt said he felt as fit as one of these animals
Bull moose
What is the largest species in the deer family that is native to North America? 

This gemlike carving material was obtained from walruses as well as elephants
ivory
What material is commonly used in the creation of piano keys and fine sculptures? 

In "Gulliver's Travels", Swift described this type of creature as "the most unteachable of all brutes"
a Yahoo
"What is the most popular search engine used in the early 2000s?" 

